In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report


호주 멜버른의 주택 가격 데이터셋 melb_data.csv으로 주택 가격을 예측하고,   
각종 평가지표를 사용하여 결정트리 모델 평가하기.

#### Data Description

* `Rooms`: 방 개수

* `Price`: 가격(단위: $ 달러)

* `Type`: 주택 유형   
br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.

* `Distance`: Distance from CBD(Central Business District : 도심 지역)

* `Bedroom2` : Scraped # of Bedrooms (from different source) (주요 침실 개수)

* `Bathroom`: Number of Bathrooms

* `Car`: Number of carspots

* `Landsize`: Land Size

* `BuildingArea`: Building Size

* `CouncilArea`: Governing council for the area


우선 다음 코드들을 실행하여 melb_data변수명으로 데이터를 로드하고   
결정트리 분석에 필요한 컬럼만 남기고 결손값을 처리합니다.      
<span style='color:green'>(로컬 환경에 알맞게 file_path를 변경하세요)</span>
      
* feature 데이터 = [Rooms, Type, Distance, Bedroom2, Bathroom, Car, Landsize, BuildingArea, CouncilArea]   
* label 데이터 = Price      
   
(Price 값이 양적 자료이기 때문에 분류보단 회귀방식의 모델이 더 적합하지만, 결정트리를 사용하기 위해 편의상 Price값을 사분위수에 따라 범주형 자료로 변환하여 사용합니다.)

In [3]:
file_path = 'melb_data.csv'
drop_columns=['Date','YearBuilt','Suburb','SellerG','Postcode','Address','Method','Lattitude','Longtitude','Propertycount']
melb_data = pd.read_csv(file_path).drop(columns=drop_columns,axis=1)
melb_data['CouncilArea'] = melb_data['CouncilArea'].fillna('Moreland')
melb_data['BuildingArea'] = melb_data['BuildingArea'].fillna(melb_data['BuildingArea'].mean())
melb_data['Car'] = melb_data['Car'].fillna(melb_data['Car'].median())

In [5]:
#melb_data.info()

LabelEncoder를 사용하여 인코딩을 합니다.

In [6]:
categorical_features = [ 'Type', 'CouncilArea','Regionname']
for col in categorical_features:
    le = LabelEncoder()
    melb_data[col] = le.fit_transform(melb_data[col])

### 전처리
label 데이터(target)로 Price 를 사용.   
아래 규칙과 같이 Price값을 바꾸기.

Q1: 25%분위 수 Q2: 50%분위 수, Q3: 75%분위 수
   
   <span style='color:red'>[ 규칙 ]</span>
* 0 이상, Q1 미만 : '0'
* Q1 이상, Q2 미만 : '1'
* Q2 이상, Q3 미만 : '2'
* Q3 이상 : '3'
      

In [7]:
def price_to_level(prices):
    Q1,Q2,Q3 = np.percentile(prices,[25,50,75])

    intervals = [(0,Q1),(Q1,Q2),(Q2,Q3),(Q3,prices.max()+1)]
    p_levels = []
    for p in prices:
        for i,interval in enumerate(intervals):
            if interval[0] <= p < interval[1]:
                p_levels.append(f'{i}')

    return p_levels

# price_to_level() 커스텀 함수를 사용하여 Price컬럼값을 변경
melb_data['Price'] = price_to_level(melb_data['Price'])
melb_data = melb_data.rename(columns={'Price':'PriceLevel'})


In [8]:
print(melb_data.loc[8,'PriceLevel'],melb_data.loc[3,'PriceLevel'],
      melb_data.loc[1,'PriceLevel'],melb_data.loc[0,'PriceLevel'])

0 1 2 3


### 정확도
train_test_split() 함수로 학습/테스트용 데이터로 분리하고,

DecisionTreeClassifier를 사용하여 모델을 학습하고 PriceLevel을 예측하기

예측 결과의 정확도(accuracy) 평가

<span style='color:green'>(random_state = 10, test_size = 0.25)</span>
* features 데이터 : PriceLevel을 제외한 나머지 컬럼   
* label 데이터 : PriceLevel 컬럼

In [9]:
features = melb_data.drop('PriceLevel',axis=1)
label = melb_data['PriceLevel']
X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.25,random_state=10)

dt_clf = DecisionTreeClassifier(random_state=10)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)

np.round(accuracy_score(y_test,pred),4)

0.6236

### 정밀도와 재현율
(pred)값과 실제값(y_test)을 토대로 confusion matrix 구현하기      


In [11]:
classes = ['0', '1', '2', '3'] # 레이블 클래스
cm = (confusion_matrix(y_test,pred))
# pandas DataFrame으로 변환
pd.DataFrame(cm,index=classes,columns=classes)

,0,1,2,3
0,649,183,22,4
1,175,434,186,45
2,32,186,450,196
3,8,31,210,584


###  직접 재현율(recall)과 정밀도(precision)를 구해서 확인

a) Macro Averaging 방식 사용

In [12]:
macro_precisions = []
macro_recalls = []
TPS = []
FPS = []
FNS = []
for i in range(cm.shape[0]):
    TP = cm[i][i]
    FP = 0
    FN = 0
    for j in range(cm.shape[0]):
        if i == j :
            continue
        FP += cm[j][i]
        FN += cm[i][j]
    macro_precisions.append(TP/(TP+FP))
    macro_recalls.append(TP/(TP+FN))
    TPS.append(TP)
    FPS.append(FP)
    FNS.append(FN)
macro_precision = np.mean(macro_precisions)
print('macro_precision:',macro_precision)
macro_recall = np.mean(macro_recalls)
print('macro_recall:',macro_recall)

macro_precision: 0.623609372215409
macro_recall: 0.6237476721457814


b) micro-averaging 방식 사용

In [13]:
micro_precision = sum(TPS)/(sum(TPS)+sum(FPS))
micro_recall = sum(TPS)/(sum(TPS)+sum(FNS))
print('micro_precision:',micro_precision)
print('micro_recall:',micro_recall)

micro_precision: 0.6235640648011782
micro_recall: 0.6235640648011782


d) sklearn.metrics의 classification_report() 함수를 사용하여 구한 평가지표 결과를 clf_report 변수에 저장  


In [14]:
clf_report = classification_report(y_test,pred,labels=classes,target_names=classes,output_dict=True)
clf_report_df = pd.DataFrame(clf_report)
clf_report_df

,0,1,2,3,accuracy,macro avg,weighted avg
precision,0.751157,0.520384,0.518433,0.704463,0.623564,0.623609,0.623375
recall,0.756410,0.516667,0.520833,0.701080,0.623564,0.623748,0.623564
f1-score,0.753775,0.518519,0.519630,0.702768,0.623564,0.623673,0.623464
support,858.000000,840.000000,864.000000,833.000000,0.623564,3395.000000,3395.000000


### 회귀 모델에 대한 평가

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# 예시 데이터 생성
np.random.seed(0)  # 결과의 재현성을 위해 시드 설정
X = 2.5 * np.random.randn(1000) + 1.5  # 독립변수 X 생성
res = 0.5 * np.random.randn(1000)       # 잔차 생성
y = 2 + 0.3 * X + res                   # 종속변수 y 생성

# 데이터프레임 형태로 변환
df = pd.DataFrame({
    'X': X,
    'y': y
})

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(df['X'], df['y'], test_size=0.2, random_state=0)

# 모델 학습
model = LinearRegression()
model.fit(X_train.values.reshape(-1, 1), y_train)

# 예측
y_pred = model.predict(X_test.values.reshape(-1, 1))

# 평가
mse = mean_squared_error(y_test, y_pred) #MSE
rmse = np.sqrt(mse)  #RMSE
mae = mean_absolute_error(y_test, y_pred)  #MAE
mape = mean_absolute_percentage_error(y_test, y_pred)  #MAPE

print(f'MSE: {np.round(mse,4)}')
print(f'RMSE:{np.round(rmse,4)}')
print(f'MAE: {np.round(mae,4)}')
print(f'MAPE: {np.round(mape,4)}')

MSE: 0.245
RMSE:0.495
MAE: 0.3776
MAPE: 0.2375
